In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [3]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [4]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

In [5]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '../../../data/raw/'

# model config
model_name = 'efficientnet_b0'

# training config
img_size = 224
LR = 5e-4
EPOCHS = 10
BATCH_SIZE = 64
num_workers = 4


In [8]:
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=0.05, 
        scale_limit=0.1, 
        rotate_limit=15, 
        p=0.3
    ),
    A.OneOf([
        A.GaussNoise(var_limit=(10, 50), mean=0, p=0.5),
        A.MotionBlur(blur_limit=3, p=0.5),
    ], p=0.2),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

/data/ephemeral/home/py310/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_21838/704947896.py:12: UserWarning: Argument(s) 'var_limit, mean' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), mean=0, p=0.5),


In [9]:
# Dataset 정의
trn_dataset = ImageDataset(
    "../../../data/raw/train.csv",
    "../../../data/raw/train/",
    transform=trn_transform
)

tst_dataset = ImageDataset(
    "../../../data/raw/sample_submission.csv",
    "../../../data/raw/test/",
    transform=tst_transform
)


In [10]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [11]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)

# loss & optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)

# scheduler (optional but strongly recommended)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)



model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

In [12]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 0.5200: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:04<00:00,  5.77it/s]


train_loss: 1.2023
train_acc: 0.6490
train_f1: 0.6335
epoch: 0.0000



Loss: 0.3686: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00, 11.07it/s]


train_loss: 0.3163
train_acc: 0.8962
train_f1: 0.8930
epoch: 1.0000



Loss: 0.2483: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00, 11.41it/s]


...

train_loss: 0.1123
train_acc: 0.9548
train_f1: 0.9508
epoch: 7.0000



Loss: 0.1759: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00, 11.39it/s]


train_loss: 0.0924
train_acc: 0.9688
train_f1: 0.9688
epoch: 8.0000



Loss: 0.0796: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00, 11.31it/s]

train_loss: 0.0988
train_acc: 0.9650
train_f1: 0.9627
epoch: 9.0000



# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [13]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  6.96it/s]


In [14]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [15]:
sample_submission_df = pd.read_csv("../../../data/raw/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [16]:
pred_df.to_csv("pred_2.csv", index=False)

In [17]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,6
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,15
4,00901f504008d884.jpg,2
